# HydroSOS Publish Status Products
#### Jose Valles (jose.valles.leon@gmail.com)

In [1]:
import os
import requests
import json 

# This code line allows to remove warning produce by verify = False
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

## Send the results to Dinagua Web Service for publishing

After creating the json file for the corresponding year-month, we will update the Web Service using the following process

In [5]:
# Locate the corresponding json file
json_file = '../waterbalance/output_json/01_month/2024-07.json'

# Define Parameters
date_name = json_file.split('/')[-1].split('.')[0]
# scale: 1, 3, 12
cat_scale = 1
# ts: Estaciones or Modelos
ts = "Modelos"

In [4]:
# Ambiente produccion
url_base = 'https://www.ambiente.gub.uy/dinaguaws/'
# Username and password
user_dinagua = {"user":"FEWS-Uruguay","password":"4MEzOeKFgp0pj9lATSIF"}
# Get the token from DINAGUA WS
response_token = requests.post(url=url_base + "gettoken", json=user_dinagua, verify=False);
response_token.close()
# Extract token and assign it to a variable
token = response_token.text
# Add the token to a bearer authorization
headers = {"Authorization" : "Bearer " + token}

POST Method for creating the filter Id

In [6]:
# create the json dictonary
dataFiltro = {"fecha": date_name,"temporalidad": cat_scale,"serietemporal": ts}

# POST Save Filter
post_filtro = requests.post(url=url_base + "estadohidro/datoscat/guardarFiltro", headers = headers, json=dataFiltro, verify=False)
post_filtro.close()
print(post_filtro.text)

Filtro guardado con éxito


GET Method for extracting the FilterID

In [ ]:
# Create the json dictonary
params_filtros = {"fecha" : date_name, "temporalidad" : cat_scale, "serietemporal" : ts}
# Send requests
get_filtro_id = requests.get(url = url_base + "estadohidro/filtros",headers=headers, params=params_filtros, verify=False)
get_filtro_id.close()
# print id
id = json.loads(get_filtro_id.content.decode('utf-8'))
id = id[0]["id"]
print(id)

POST HydroSOS Status for the corresponding Month/Year

In [ ]:
# Creating the Request Body
datosCat = {"filtrosId": {id}}
# importing the json file from the output directory
json_body = open(json_file)
json_body = json.load(json_body)
# POST HydroSOS status 
post_estado = requests.post(url=url_base+"estadohidro/datoscat/guardarDatosCat",headers=headers,params=datosCat,json=json_body,verify=False)
post_estado.close()
print(post_estado.text)

We are going to see if the hydrological status is correctly added in the service

In [ ]:
params_status = {"serietemporal": ts, "fecha": date_name, "temporalidad": cat_scale}
get_status = requests.get(url=url_base+"estadohidro/datoscat",headers=headers,params=params_status,verify=False)
get_status.close()
# Convert it to json and dataframe
response_status = json.loads(get_status.content.decode('utf-8'))

### Calculate the Accumulated Hydrological Status for 3-Months (Quaterly)

After testing the HydroSOS Package and sending the results to Dinagua Web Service, we calculate the accumulated status product

In [ ]:
DISCHARGE_THREE_MONTHS = aggvar.calculate_accumulated(flowdata.reset_index(), 3)
HTML(DISCHARGE_THREE_MONTHS.tail(6).to_html())

In [ ]:
QUATERLY_STATUS = SOS.quarterly_status(DISCHARGE_THREE_MONTHS)
HTML(QUATERLY_STATUS.tail(6).to_html())

### Calculate the Accumulated Hydrological Status for 12-Months (Annualy)

After testing the HydroSOS Package and sending the results to Dinagua Web Service, we calculate the accumulated status product

In [ ]:
DISCHARGE_TWELVE_MONTHS = aggvar.calculate_accumulated(flowdata.reset_index(), 12)
HTML(DISCHARGE_TWELVE_MONTHS.tail(6).to_html())

In [ ]:
DISCHARGE_TWELVE_MONTHS

In [ ]:
ANNUALY_STATUS = SOS.annualy_status(DISCHARGE_TWELVE_MONTHS)
HTML(ANNUALY_STATUS.tail(6).to_html())